In [488]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from collections.abc import Iterable

In [487]:
iris = load_iris()
X = iris.data
y = iris.target
df = pd.DataFrame(X, columns=iris.feature_names)
df["Class"] = y


In [4]:
df.describe()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [553]:
class Node:
    def __init__(self, model_class):
        # self.X = X
        # self.y = y
        # self.gini = gini
        self.model_class = model_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [567]:
class MyDecisionTreeClassifier:
    
    def __init__(self, max_depth):
        self.max_depth = max_depth
    
    def gini(self, sample_distr, num_samples):
        '''
        A Gini score gives an idea of how good a split is by how mixed the
        classes are in the two groups created by the split.
        
        A perfect separation results in a Gini score of 0,
        whereas the worst case split that results in 50/50
        classes in each group result in a Gini score of 0.5
        (for a 2 class problem).
        '''
        gini = 1 - sum((n / num_samples)**2 for n in sample_distr)
        return gini


    def split_data(self, X, y) -> tuple[int, int]:
        
        # test all the possible splits in O(N^2)
        # return index and threshold value
        
        node_num_samples = y.size
        print(y.size)
        if node_num_samples <= 1:
            return None, None
        samples_dstr_parent = [np.sum(y == c) for c in range(self.num_classes)]
        best_gini = self.gini(samples_dstr_parent,node_num_samples)
        best_feature, best_threshold = None, None
        for feature_idx in range(self.num_features):
            thresholds, classes = zip(*sorted(zip(X[:, feature_idx], y)))
            left_samples_dstr = [0]*self.num_classes
            right_samples_dstr = samples_dstr_parent.copy()
            for i in range(1,node_num_samples):
                c = classes[i-1]
                left_samples_dstr[c] += 1
                right_samples_dstr[c] -= 1
                gini_left = 1.0 - sum(
                    (left_samples_dstr[x] / i) ** 2 for x in range(self.num_classes)
                )
                gini_right = 1.0 - sum(
                    (right_samples_dstr[x] / (node_num_samples - i)) ** 2 for x in range(self.num_classes)
                )
                gini_child = (i * gini_left + (node_num_samples - i) * gini_right) / node_num_samples
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini_node < best_gini:
                    best_gini = gini_node
                    best_feature = feature_idx
                    best_threshold = (thresholds[i] + thresholds[i - 1]) / 2
        return best_feature, best_threshold
    
    def build_tree(self, X, y, depth = 0):
        # create a root node
        # recursively split until max depth is not exeeced
        samples_dstr = [np.sum(y == i) for i in range(self.num_classes)]
        model_class = np.argmax(samples_dstr)
        node = Node(model_class)
        if depth < self.max_depth:
            feature_idx, threshold = self.split_data(X,y)
            if feature_idx:
                left_tree_mask = X[:,feature_idx] < threshold
                X_left, y_left = X[left_tree_mask], y[left_tree_mask]
                X_right, y_right = X[~left_tree_mask], y[~left_tree_mask]
                node.feature_index = feature_idx
                node.threshold = threshold
                node.left = self.build_tree(X_left, y_left, depth + 1)
                node.right = self.build_tree(X_right, y_right, depth + 1)
        return node
        
    def fit(self, X, y):
        # basically wrapper for build tree
        self.num_classes = len(set(y))
        self.num_features = X.shape[1]
        self.tree = self.build_tree(X,y)
    
    def _predict(self,X_test):
        node = self.tree
        while node.left:
                if X_test[node.feature_index] < node.threshold:
                    node = node.left
                else:
                    node = node.right
        return node.model_class

    def predict(self, X_test):
        # traverse the tree while there is left node
        # and return the predicted class for it, 
        # note that X_test can be not only one example
        node = self.tree
        if isinstance(X_test[0],Iterable):
            models = []
            for test in X_test:
                models.append(self._predict(test))
            return models
        else:
            return self._predict(X_test)


In [568]:
from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
ctree = MyDecisionTreeClassifier(10)
X, X_test, y, y_test = train_test_split(X, y, test_size= 0.25)
ctree.fit(X,y)

112


NameError: name 'node_nusamplples' is not defined

In [560]:
from sklearn.metrics import mean_squared_error
models = ctree.predict(X_test)
print(mean_squared_error(y_test,models))
sum(models == y_test) / len(y_test)


AttributeError: 'MyDecisionTreeClassifier' object has no attribute 'tree'

In [159]:
cart.tree.right.threshold


1.75